In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
import time

# Ajuster les paramètres pour afficher toutes les lignes et colonnes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)


In [4]:
import copy
from collections import Counter

#colonne cible type
cols_cible_type = ["TurnoutTimeSeconds", "TravelTimeSeconds", "PumpSecondsOnSite"]
#colonne Data par colonne cible type
cols_Data = [
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "PropertyType", "StopCode"],
]
#colonne cible bins
# utilise -1 sinon crée bizarrement des Nan pour les valeurs à 0 au lieu de mettre 0
cols_cible_bins = [
    {
        "bins": np.array([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20000]) * 60,
        "labels": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 30],
    },
    {
        "bins": np.array([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20000]) * 60,
        "labels": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 30],
    },
    {
        "bins": np.array([-1, 5, 10, 15, 30, 45, 60, 120, 180, 360, 1000000]) * 60,
        "labels": [5, 10, 15, 30, 45, 60, 120, 180, 360, 1000],
    },
]

cols_cible_bins_numbers = [4, 10, 9]
#colonne cible avec (min, mean, max) consideration
cols_cible = [
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"],
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"]
]

# copie profonde, sinon la simple copie fait une copie des références des sous tableaux, et leur changement modifie l'original
cols_cible_minutes = copy.deepcopy(cols_cible)


def load_df(col_cible_type):
    """
    - Charge le dataset de la colonne cible type (_df_ready_{col_cible_type}.csv).
    
    Args:
       col_cible_type : nom de la colonne cible type

    Returns:
        pd.DataFrame: Le DataFrame contenant les données de la colonne cible type
    """
    df = pd.read_csv(
        f"../data/_df_ready_{col_cible_type}.csv", sep=";", low_memory=False
    )
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df


def load_df_full(col_cible_type):
    """
    - Charge le dataset de la colonne cible type (_df_full_{col_cible_type}.csv).
    
    Args:
       col_cible_type : nom de la colonne cible type

    Returns:
        pd.DataFrame: Le DataFrame contenant les données de la colonne cible type
    """
    df = pd.read_csv(
        f"../data/_df_ready_full_{col_cible_type}.csv", sep=";", low_memory=False
    )
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df
     
def create_cols_cible_minutes(df, index, bins=-1):
    """
    Crée de nouvelles colonnes cible basées sur des tranches en minutes pour une dataframe donnée.

    Paramètres :
    df (pandas.DataFrame) : La dataframe contenant les données.
    index (int) : L'index correspondant au type de cible à transformer.
    bins (int) : Le nombre de tranches (par défaut -1, utilise les valeurs prédéfinies).

    Retourne :
    pandas.DataFrame : La dataframe avec les nouvelles colonnes cible en minutes.
    """
    # Crée 3 nouvelles target pour chaque min/mean/max, par tranche en minutes, pour la cible type actuelle
    for index_cible, col_cible in enumerate(cols_cible[index]):
        # bins = cols_cible_bins[index]["bins"]
        # print(bins)
        # labels = cols_cible_bins[index]["labels"]
        # print(labels)

        #Genere un nouveau nom pour chaque colonne cible en remplacant Seconds par Minutes
        new_name = col_cible.replace("Seconds", "Minutes")
        print(new_name, col_cible)

        #Determine le nombre de tranches(bins) 
        #soit par la valeur prédéfinies, soit par la valeur pasée en parametre
        num_bins = cols_cible_bins_numbers[index] if bins == -1 else bins
        new_bins = pd.qcut(df[col_cible], q=num_bins, precision=0)

        # Prend la valeur haute de l'intervalle pour la nouvelle colonne
        df[new_name] = new_bins.apply(lambda x: x.right) 
    
        # 3. Affiche la répartition des observations dans chaque bin
        # bin_counts = df[new_name].value_counts().sort_index()
        # print(bin_counts)
    
        #Met à jour la liste des nouvelles colonnes cible en minute
        cols_cible_minutes[index][index_cible] = new_name
        print("cols_cible", cols_cible)
        print("cols_cible_minutes", cols_cible_minutes)

    #Affiche les premieres lignes du dataframe mise à jour
    display(df.head(3))

    #Retourne le dataframe mise à jour
    return df

def Create_X(df_limited, index):
    """
    Crée un DataFrame X sans les colonnes cibles et ne conservant que certaines colonnes explicatives.

    Paramètres :
    df_limited (pandas.DataFrame) : La dataframe contenant les données initiales.
    index (int) : L'index correspondant au type de données à utiliser.

    Retourne :
    pandas.DataFrame : La dataframe contenant uniquement les colonnes explicatives sélectionnées.
    """
    #Affiche la dimension du dataframe initial
    print(df_limited.shape)

    # Crée X sans les variables cibles
    # Crée une liste des colonnes à supprimer (cibles et NumPumpsAttending)
    cols_to_remove = (
        [item for sublist in cols_cible for item in sublist]
        + cols_cible_minutes[index]
        + ["NumPumpsAttending"]
    )
    print("cols_to_remove", cols_to_remove)
   # Supprime les colonnes spécifiées du dataframe 
    X = df_limited.drop(cols_to_remove, axis=1)

    # Ne conserve que certaines colonnes explicatives
    # Crée une liste des colonnes à conserver
    cols_to_keep = [
        col
        for col in X.columns
        if any(substring in col for substring in cols_Data[index])
    ]
    print("cols_to_keep", cols_to_keep)
    # Conserver uniquement ces colonnes spécifiées
    X = X[cols_to_keep]
    # Affiche les deux premières lignes du dataframe résultant
    display(X.head(2))
    # Affiche la dimension du dataframe résultant
    print(X.shape)
    
    return X

def under_sampling(X_train, y_train):
      """
    Réalise un suréchantillonnage des données d'entraînement en utilisant la méthode SMOTE.

    Paramètres :
    X_train (pandas.DataFrame) : Les caractéristiques d'entraînement.
    y_train (pandas.Series) : Les étiquettes d'entraînement.

    Retourne :
    tuple : Un tuple contenant les caractéristiques et les étiquettes suréchantillonnées.
    """
    # Réchantillonne
    print("Resampling")
    print(len(X_train))
    # Enregistre le temps de début
    start_time = time.time()
    print(y_train.value_counts())
    # sampling = RandomUnderSampler(sampling_strategy="auto", random_state=42)
    
    # Crée l'objet SMOTE pour le suréchantillonnage
    print("Smote")
    sampling = SMOTE(random_state=42)
    X_train_samp, y_train_samp = sampling.fit_resample(X_train, y_train)
    # Affiche la répartition initiale des étiquettes
    print(y_train_samp.value_counts())

    # print("EditedNearestNeighbours")
    # sampling = EditedNearestNeighbours(n_jobs=-1)
    # X_train_samp, y_train_samp = sampling.fit_resample(X_train_samp, y_train_samp)
    
    # print("ClusterCentroids")
    # sampling = ClusterCentroids(random_state=42)
    # X_train_samp, y_train_samp = sampling.fit_resample(X_train, y_train) # 35 minutes sur 3 cibles


    # print("RandomUnderSampler à 60%")
    # # Compte la taille initiale des classes avant SMOTE
    # class_counts_before_smote = Counter(y_train)
    # max_class_count_before_smote = max(class_counts_before_smote.values())
    # # Calcule 60% de la taille de la classe majoritaire initiale pour chaque classe pour appliquer le RandomUnderSampler
    # sampling_strategy = {label: int(0.60 * max_class_count_before_smote) for label in class_counts_before_smote}
    # # Appliquer RandomUnderSampler pour ajuster les classes 
    # undersampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
    # X_train_samp, y_train_samp = undersampler.fit_resample(X_train_samp, y_train_samp)
    
    # Enregistre le temps de fin
    end_time = time.time()
    
    # Affiche la nouvelle répartition des étiquettes après suréchantillonnage
    print(y_train_samp.value_counts())
    
    # Affiche la durée du suréchantillonnage
    print(f"Durée de SMOTE : {end_time - start_time:.2f} secondes")
    print(len(X_train_samp))
    
    # Retourne les données suréchantillonnées
    return X_train_samp, y_train_samp

In [16]:
# --------------------------------------- Pytorch régression
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torchsummary import summary
from torch import optim
from tqdm import tqdm

torch.cuda.is_available()

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, accuracy_score, f1_score

from sklearn.metrics import (
    root_mean_squared_error,
    mean_absolute_error,
    r2_score,
)

import seaborn as sns
import matplotlib.pyplot as plt

#Colonnes cibles
cols_cible = [
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"],
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"],
]
#Filtre sur les colonnes cibles(xxx_mean)
cols_cible_filter = [
    "TurnoutTimeSeconds_mean",
    "TravelTimeSeconds_mean",
    "PumpSecondsOnSite_mean",
]
# Détermine si le GPU est disponible, sinon utilise le CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device utilisé :", device)
# Definir Taille du lot d'entraînement
train_batch_size = 512

class CustomDataset(torch.utils.data.Dataset):
    """
    Classe personnalisée pour les données d'entraînement.
    
    Paramètres :
    X (pandas.DataFrame) : Les caractéristiques (features) des données.
    y (pandas.Series) : Les étiquettes (labels) des données.
    """

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, idx):
        """
        Retourne une paire (x, y) pour un index donné.
        
        Paramètres :
        idx (int) : L'index de la donnée.
        
        Retourne :
        list : Une liste contenant la caractéristique (x) et l'étiquette (y).
        """
        # Pour chaque donnée, retourner [x, y]
        x = self.X[idx]
        y = self.y[idx]
        return [x, y]

    def __len__(self):
        """
        Retourne la longueur du dataset.
        
        Retourne :
        int : Le nombre total de données dans le dataset.
        """
        return len(self.X)

def get_model():
    """
    Crée et retourne un modèle séquentiel de réseau de neurones.

    Retourne :
    torch.nn.Sequential : Le modèle de réseau de neurones.
    """
    #Crée un modèle séquentiel de réseau de neurones avec plusieurs couches linéaires et des fonctions d'activation ReLU.
    model = nn.Sequential(
        nn.Linear(X.shape[1], 100),
        nn.ReLU(),
        nn.Linear(100, 100),
        nn.ReLU(),
        nn.Linear(100, 100),
        nn.ReLU(),
        nn.Linear(100, 50),
        nn.ReLU(),
        nn.Linear(50, 1),
    )
    #Le modèle est déplacé vers le dispositif approprié (GPU ou CPU)
    model.to(device)
    #affiche un résumé du modèle avec la taille de l'entrée spécifiée
    #par input_size=(X.shape[-1],) et le dispositif device
    summary(model, input_size=(X.shape[-1],), device=device)
    #retourne le model créé
    return model

def train_model(model, train_loader):
    """
    Entraîne le modèle de réseau de neurones sur les données fournies.

    Paramètres :
    model (torch.nn.Module) : Le modèle de réseau de neurones à entraîner.
    train_loader (torch.utils.data.DataLoader) : Le DataLoader contenant les données d'entraînement.

    Retourne :
    torch.nn.Module : La fonction de perte utilisée pour l'entraînement.
    """
    epochs = 100
    # Définition de l'optimizer
    optimizer = optim.Adam(model.parameters(), 1e-3)
    # Définition de la fonction de perte
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        # Dans ce mode certaines couches du modèle agissent différemment
        #entrainement du modele
        model.train()
        loss_total = 0
        with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}", position=0, leave=True) as pbar_epoch:
            for batch_idx, batch in enumerate(train_loader):
                with tqdm(total=train_batch_size, desc=f"Batch {batch_idx+1}", position=1, leave=False) as pbar_batch:
                    # Recupere le Batch de données
                    X_batch, y_batch = batch
                    # Envoi les données au device (GPU ou CPU)
                    X_batch = X_batch.to(device)
                    y_batch = y_batch.to(device)
                    # Gradient mis à 0
                    model.zero_grad()
                    # Calcul de prédiction
                    y_pred = model(X_batch.to(torch.float32))[:,0]
                    # Calcul de la fonction de perte
                    loss =  criterion(y_pred*100, y_batch.to(torch.float32)*100) #torch.mean(torch.abs(y_pred- y_batch.to(torch.float32)))#
                    # Backpropagation : calculer le gradient de la loss en fonction de chaque couche
                    loss.backward()
                    # Clipper le gradient entre 0 et 1 pour plus de stabilité
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    # Descente de gradient : actualisation des paramètres
                    optimizer.step()
                    loss_total += loss.item()
                    # Mise à jour des bars de progression pour suivre l'avancement de l'epoch
                    pbar_batch.update(train_batch_size)
                    pbar_epoch.update(1)

    #Affiche la perte moyenne d'entrainement  Training loss
    print(f"Epoch : {epoch+1}/{epochs} -- Training loss {loss_total/len(train_loader)}")
    #retourne la fonction de perte a la fin de l'entrainement
    return criterion

def predict(dataloader_val, criterion):
    """
    Évalue le modèle sur les données de validation et calcule la perte moyenne.

    Paramètres :
    dataloader_val (torch.utils.data.DataLoader) : Le DataLoader contenant les données de validation.
    criterion (torch.nn.Module) : La fonction de perte utilisée pour évaluer le modèle.

    Retourne :
    tuple : Un tuple contenant la perte moyenne, les prédictions et les vraies valeurs.
    """
    # Passer le modèle en évaluation
    model.eval()
    # Calculer la loss totale
    loss_val_total = 0
    # Stocker les prédictions et les vraies valeurs.
    predictions, true_vals = [], []
    for batch in dataloader_val:
        X_batch, y_batch = batch
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        with torch.no_grad():
            # Prédiction du modèle pour un batch donné
            y_pred = model(X_batch.to(torch.float32))
        # Calcul de la fonction de perte pour l'utiliser comme une métrique
        loss = criterion(y_pred, y_batch.to(torch.float32))
        # Cumuler la fonction de perte de tous les lots de données.
        loss_val_total += loss.item()
        # Enregistrer les prédictions pour les utiliser plus tard
        predictions.extend(y_pred.detach().cpu().numpy())
        # Enregistrer les vraies valeurs pour les utiliser plus tard
        true_vals.extend(y_batch.cpu().numpy())

    # Loss du jeu de données val
    loss_val_avg = loss_val_total / len(dataloader_val)
    # Ensemble des prédictions du jeu de données
    predictions = np.array(predictions)
    # Ensemble des vraies valeurs du jeu de données
    true_vals = np.array(true_vals)
    
    return loss_val_avg, predictions, true_vals

# Les résultats de chaque itération sont stockés dans la liste results et sauvegardés dans un fichier CSV.
results = []
# Crée un objet KFold pour réaliser une validation croisée en 5 plis avec les données mélangées.
crossval = KFold(n_splits=5, random_state=42, shuffle=True)

for index, name in enumerate(cols_cible_type):
    #Chargement et préparation des données pour chaque type de cible
    df = load_df(name)

    for year_floor in range(14, 15):  # >= 2022
        print("-----------------------------------------------------")
        print("year", 2024 - 16 + year_floor)
        print("-----------------------------------------------------")
        #df_limited : les données sont limitées à une certaine année
        df_limited = df[df.CalYear >= year_floor]
        #definir la variable cible
        X = Create_X(df_limited, index).values

        for index_cible, col_cible in enumerate(cols_cible[index]):
            if not (col_cible in cols_cible_filter):
                continue
            print(
                "--------------------------------------------------------------------------------"
            )
            print("///////////////////////////", "cible", col_cible)
            # Pour chaque cible, les étiquettes (y) sont extraites 
            # et les données sont divisées en ensembles d'entraînement et de test avec train_test_split.
            y = df_limited[col_cible].values
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42
            )

            # Transforme train et test pour Pytorch
            train_set = CustomDataset(X_train, y_train)
            train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
            test_set = CustomDataset(X_test, y_test)
            test_loader = DataLoader(test_set, batch_size=256, shuffle=False)
            print(next(iter(train_loader)))
            print(next(iter(test_loader)))
            # Crée le modèle
            model = get_model()
            # Entraine le model
            criterion = train_model(model, train_loader)
            # évalue le modèle
            loss, y_train_pred, y_train_true = predict(train_loader, criterion)
            loss, y_pred, y_true = predict(test_loader, criterion)
            print(f"Loss: {loss}")
            #Calcul des metriques RMSE
            rmse_train = root_mean_squared_error(y_train_true, y_train_pred)
            print(f"RMSE train : {rmse_train}")
            rmse_test = root_mean_squared_error(y_true, y_pred)
            print(f"RMSE test : {rmse_test}")
            # Calcul des metriques MAE
            mae_train = mean_absolute_error(y_train_true, y_train_pred)
            print(f"MAE train : {mae_train}")
            mae_test = mean_absolute_error(y_true, y_pred)
            print(f"MAE test : {mae_test}")
            # Ajouter au tableau des résultats
            results.append(
                {
                    "Target": col_cible,
                    "Year floor": 2024 - 16 + year_floor,
                    "Loss": loss,
                    "RMSE train": rmse_train,
                    "RMSE test": rmse_test,
                    "MAE train": mae_train,
                    "MAE test": mae_test,
                }
            )
            #Les résultats de chaque itération sont stockés dans la liste results .
            df_results = pd.DataFrame(results)
            df_results = df_results.sort_values(
                by=["Target", "MAE test"], ascending=[False, True]
            )
            #Sauvegarde dans un fichier CSV.
            df_results.to_csv(
                f"../data/_Pytorch regression.csv", sep=";", index=False
            )

print(results)
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by=["Target", "MAE test"], ascending=[False, True])
df_results.to_csv(f"../data/_Pytorch regression.csv", sep=";", index=False)

Device utilisé : cuda
-----------------------------------------------------
year 2022
-----------------------------------------------------
(342634, 42)
cols_to_remove ['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max', 'TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max', 'PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max', 'TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max', 'NumPumpsAttending']
cols_to_keep ['CalYear', 'HourOfCall_0', 'HourOfCall_1', 'HourOfCall_2', 'HourOfCall_3', 'HourOfCall_4', 'Postcode_district_0', 'Postcode_district_1', 'Postcode_district_2', 'Postcode_district_3', 'Postcode_district_4', 'Month_0', 'Month_1', 'Month_2', 'Month_3', 'DayOfWeek_0', 'DayOfWeek_1', 'DayOfWeek_2']


,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2
1248893,14,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,1
1248894,14,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,1


(342634, 18)
--------------------------------------------------------------------------------
/////////////////////////// cible TurnoutTimeSeconds_mean
[tensor([[14,  1,  0,  ...,  1,  1,  0],
        [16,  0,  0,  ...,  0,  1,  1],
        [14,  1,  0,  ...,  1,  1,  0],
        ...,
        [15,  0,  0,  ...,  0,  1,  1],
        [15,  0,  1,  ...,  0,  1,  1],
        [14,  0,  0,  ...,  0,  1,  1]]), tensor([ 53.0000,  77.0000,  56.0000,  40.0000, 333.0000,  28.5000,  76.0000,
         90.0000,  57.0000,  10.0000, 135.0000, 121.0000,  45.0000,  44.0000,
        111.0000,  63.0000,  71.0000,  29.0000,  77.0000,  50.5000,  26.0000,
        133.5000,  65.0000,   9.0000,  60.0000,  88.0000,  72.0000,  63.0000,
         68.0000,  55.0000,  71.0000, 128.0000, 121.0000,  85.0000,  62.0000,
         99.0000,  67.0000,  32.0000,  39.0000,  17.0000, 145.0000,  83.0000,
         64.0000, 118.0000,  77.0000,  59.0000,  64.0000,  46.5000,  65.0000,
        143.0000, 111.0000,  50.0000,  63.5000

Epoch 100/100: 100%|██████████| 536/536 [00:07<00:00, 70.13it/s]
c:\Users\lordb\anaconda3\envs\projet-IA-312\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch : 100/100 -- Training loss 11703206.333955225


...

Epoch 100/100: 100%|██████████| 536/536 [00:07<00:00, 70.51it/s]
c:\Users\lordb\anaconda3\envs\projet-IA-312\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch : 100/100 -- Training loss 16875108454.208956


c:\Users\lordb\anaconda3\envs\projet-IA-312\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([187])) that is different to the input size (torch.Size([187, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\lordb\anaconda3\envs\projet-IA-312\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Loss: 2652929.0475746267
RMSE train : 1296.6751493869788
RMSE test : 1294.5039316807713
MAE train : 702.3303524746951
MAE test : 707.3933636893315
[{'Target': 'TurnoutTimeSeconds_mean', 'Year floor': 2022, 'Loss': 1598.6247834162925, 'RMSE train': 34.18922020415068, 'RMSE test': 34.473754602898055, 'MAE train': 22.002475214264933, 'MAE test': 22.068980878012514}, {'Target': 'TravelTimeSeconds_mean', 'Year floor': 2022, 'Loss': 20641.210256092585, 'RMSE train': 127.5260806843354, 'RMSE test': 128.2096122014373, 'MAE train': 94.02023373596421, 'MAE test': 94.26319705983676}, {'Target': 'PumpSecondsOnSite_mean', 'Year floor': 2022, 'Loss': 2652929.0475746267, 'RMSE train': 1296.6751493869788, 'RMSE test': 1294.5039316807713, 'MAE train': 702.3303524746951, 'MAE test': 707.3933636893315}]


c:\Users\lordb\anaconda3\envs\projet-IA-312\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([175])) that is different to the input size (torch.Size([175, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
